In [99]:
import pandas as pd
from datetime import date

In [100]:
raw_filename = "raw_poker_results.csv"

In [101]:
raw_df = pd.read_csv(raw_filename)
raw_df.head()

,Date,Host,Notes,1,2,3,4,5,6,7,8,9,10,11,12,Unavailable
0,2021/11/23,Marlan,NaN,Jacques,Nick,Kevin,Marlan,David,Andrew,Graham,Hilton,James,Graham,NaN,NaN,NaN
1,2021/11/30,Jacques,NaN,Graham,Kevin,Jacques,Nick,Marlan,Hilton,Nick,NaN,NaN,NaN,NaN,NaN,"James, Andrew, David"
2,2021/12/07,Graham,NaN,Kevin,Hilton,Marlan,Graham,Andrew,David,Nick,Jacques,James,Nick,James,NaN,NaN
3,2021/12/14,Kevin,NaN,Graham,Kevin,Nick,Jacques,Marlan,David,Hilton,Andrew,NaN,NaN,NaN,NaN,James
4,2021/12/21,NaN,Online Evening,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
results_df = raw_df.drop(columns=["Host", "Notes", "Unavailable"])
results_df.head()

,Date,1,2,3,4,5,6,7,8,9,10,11,12
0,2021/11/23,Jacques,Nick,Kevin,Marlan,David,Andrew,Graham,Hilton,James,Graham,NaN,NaN
1,2021/11/30,Graham,Kevin,Jacques,Nick,Marlan,Hilton,Nick,NaN,NaN,NaN,NaN,NaN
2,2021/12/07,Kevin,Hilton,Marlan,Graham,Andrew,David,Nick,Jacques,James,Nick,James,NaN
3,2021/12/14,Graham,Kevin,Nick,Jacques,Marlan,David,Hilton,Andrew,NaN,NaN,NaN,NaN
4,2021/12/21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
results_df["Date"] = pd.to_datetime(results_df["Date"])
results_df.head()

,Date,1,2,3,4,5,6,7,8,9,10,11,12
0,2021-11-23,Jacques,Nick,Kevin,Marlan,David,Andrew,Graham,Hilton,James,Graham,NaN,NaN
1,2021-11-30,Graham,Kevin,Jacques,Nick,Marlan,Hilton,Nick,NaN,NaN,NaN,NaN,NaN
2,2021-12-07,Kevin,Hilton,Marlan,Graham,Andrew,David,Nick,Jacques,James,Nick,James,NaN
3,2021-12-14,Graham,Kevin,Nick,Jacques,Marlan,David,Hilton,Andrew,NaN,NaN,NaN,NaN
4,2021-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
results_df = results_df.melt(id_vars=["Date"], var_name="Position", value_name="Player").dropna()
results_df["Position"] = pd.to_numeric(results_df["Position"])
results_df = results_df.sort_values(["Date", "Position"])
results_df

,Date,Position,Player
0,2021-11-23,1,Jacques
55,2021-11-23,2,Nick
110,2021-11-23,3,Kevin
165,2021-11-23,4,Marlan
220,2021-11-23,5,David
...,...,...,...
219,2022-12-06,4,Nick
274,2022-12-06,5,Marlan
329,2022-12-06,6,James
384,2022-12-06,7,David


In [105]:
results_df = results_df.assign(Player=results_df["Player"].str.split(",")).explode("Player")
results_df["Player"] = results_df["Player"].str.strip()
results_df

,Date,Position,Player
0,2021-11-23,1,Jacques
55,2021-11-23,2,Nick
110,2021-11-23,3,Kevin
165,2021-11-23,4,Marlan
220,2021-11-23,5,David
...,...,...,...
219,2022-12-06,4,Nick
274,2022-12-06,5,Marlan
329,2022-12-06,6,James
384,2022-12-06,7,David


In [106]:
results_df = results_df.reset_index().drop(columns="index")
results_df

,Date,Position,Player
0,2021-11-23,1,Jacques
1,2021-11-23,2,Nick
2,2021-11-23,3,Kevin
3,2021-11-23,4,Marlan
4,2021-11-23,5,David
...,...,...,...
415,2022-12-06,4,Nick
416,2022-12-06,5,Marlan
417,2022-12-06,6,James
418,2022-12-06,7,David


In [107]:
results_df["Position"] = results_df.groupby("Date")["Position"].rank(method="min").astype("Int64")
results_df

,Date,Position,Player
0,2021-11-23,1,Jacques
1,2021-11-23,2,Nick
2,2021-11-23,3,Kevin
3,2021-11-23,4,Marlan
4,2021-11-23,5,David
...,...,...,...
415,2022-12-06,5,Nick
416,2022-12-06,6,Marlan
417,2022-12-06,7,James
418,2022-12-06,8,David


In [108]:
clean_raw_df = pd.pivot_table(results_df, index="Date", columns="Position", values="Player", aggfunc=lambda x: ", ".join(x))
clean_raw_df

Position,1,2,3,4,5,6,7,8,9,10,11,12
Date,,,,,,,,,,,,
2021-11-23,Jacques,Nick,Kevin,Marlan,David,Andrew,Graham,Hilton,James,Graham,NaN,NaN
2021-11-30,Graham,Kevin,Jacques,Nick,Marlan,Hilton,Nick,NaN,NaN,NaN,NaN,NaN
2021-12-07,Kevin,Hilton,Marlan,Graham,Andrew,David,Nick,Jacques,James,Nick,James,NaN
2021-12-14,Graham,Kevin,Nick,Jacques,Marlan,David,Hilton,Andrew,NaN,NaN,NaN,NaN
2022-01-11,Nick,Kevin,David,Marlan,Jacques,James,Graham,NaN,NaN,NaN,NaN,NaN
2022-01-18,Kevin,Jacques,Nick,Marlan,Hilton,Andrew,David,David,Andrew,NaN,NaN,NaN
2022-01-25,Graham,Kevin,Jacques,Nick,James,Marlan,Andrew,David,Hilton,NaN,NaN,NaN
2022-02-01,Jacques,Graham,Andrew,David,Nick,Kevin,Marlan,James,NaN,NaN,NaN,NaN
2022-02-08,Nick,Jacques,"Kevin, James",NaN,Marlan,Hilton,Andrew,Marlan,NaN,NaN,NaN,NaN


In [109]:
clean_raw_df.to_csv("clean_poker_results.csv")

In [110]:
results_pivot = pd.pivot_table(results_df, index="Player", columns="Position", aggfunc="count").astype("Int64").reset_index()
results_pivot

Player  Date                                                        \
Position              1     2     3     4     5     6     7     8     9    10   
0          Andrew     3     2     5  <NA>     6     8     9     7     5     3   
1           David     2     6     2     4     6     8     4     8     4     2   
2          Graham     6     6     3     4     6  <NA>     7     3  <NA>     2   
3          Hilton     5     4     5     4     5     7     7     2     1  <NA>   
4         Jacques     9     3     6     4     5     2     6     4     6     1   
5           James     6     1     7     2     6     8     4     4     3  <NA>   
6           Kevin     4    12    13     8     3     4     5     7     2     4   
7          Marlan     8     9     2     9     7     5     2     2     1     1   
8            Nick     4     4     9     8     3     4     6     1     3     3   
9           Peter  <NA>  <NA>  <NA>     1  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>   

                      
Position    11    12  
0            1  <NA>  
1            1  <NA>  
2         <NA>  <NA>  
3         <NA>  <NA>  
4            1  <NA>  
5            1     1  
6            2  <NA>  
7            1  <NA>  
8         <NA>  <NA>  
9         <NA>  <NA>

In [111]:
results_pivot.columns = [col[0] if not col[1] else col[1] for col in results_pivot.columns]
results_pivot

,Player,1,2,3,4,5,6,7,8,9,10,11,12
0,Andrew,3,2,5,<NA>,6,8,9,7,5,3,1,<NA>
1,David,2,6,2,4,6,8,4,8,4,2,1,<NA>
2,Graham,6,6,3,4,6,<NA>,7,3,<NA>,2,<NA>,<NA>
3,Hilton,5,4,5,4,5,7,7,2,1,<NA>,<NA>,<NA>
4,Jacques,9,3,6,4,5,2,6,4,6,1,1,<NA>
5,James,6,1,7,2,6,8,4,4,3,<NA>,1,1
6,Kevin,4,12,13,8,3,4,5,7,2,4,2,<NA>
7,Marlan,8,9,2,9,7,5,2,2,1,1,1,<NA>
8,Nick,4,4,9,8,3,4,6,1,3,3,<NA>,<NA>
9,Peter,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [112]:
results_pivot = results_pivot.sort_values(list(range(1, 13)), ascending=False).reset_index().drop(columns="index")
results_pivot

,Player,1,2,3,4,5,6,7,8,9,10,11,12
0,Jacques,9,3,6,4,5,2,6,4,6,1,1,<NA>
1,Marlan,8,9,2,9,7,5,2,2,1,1,1,<NA>
2,Graham,6,6,3,4,6,<NA>,7,3,<NA>,2,<NA>,<NA>
3,James,6,1,7,2,6,8,4,4,3,<NA>,1,1
4,Hilton,5,4,5,4,5,7,7,2,1,<NA>,<NA>,<NA>
5,Kevin,4,12,13,8,3,4,5,7,2,4,2,<NA>
6,Nick,4,4,9,8,3,4,6,1,3,3,<NA>,<NA>
7,Andrew,3,2,5,<NA>,6,8,9,7,5,3,1,<NA>
8,David,2,6,2,4,6,8,4,8,4,2,1,<NA>
9,Peter,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [113]:
results_df.to_csv("poker_results_long.csv", index=False)

In [114]:
results_pivot.to_csv("poker_results_pivot.csv", index=False)